In [23]:
def get_star_transitions(first):
    train_core = []
    test_core = []
    # first 4 nodes (1,2,3,4) in the automaton are connected to the center node

    for i in range(1,5):
        for j in range(i,3+i):
            # start, center, end
            # if i = 2 and first = 0, then we have tuples:
            # (2,5,8),(2,5,9),(2,5,6)
            train_core.append((first+i,first+5,first+6+j%4))
        # for test we would have (2,5,7)
        test_core.append((first+i,first+5,first+i+5))
    return train_core,test_core

train_core = {}
test_core = {}
# create 3 star automata
for i in range(0,3):
    train,test = get_star_transitions(i*9)
    train_core[i+1]=train
    test_core[i+1]= test

automaton2states = {}
# for each automaton, we want to create two states
for i in range(0,9*3):
    automaton2states[i+1] = (i*2+1,i*2+2)
automaton2states

{1: (1, 2),
 2: (3, 4),
 3: (5, 6),
 4: (7, 8),
 5: (9, 10),
 6: (11, 12),
 7: (13, 14),
 8: (15, 16),
 9: (17, 18),
 10: (19, 20),
 11: (21, 22),
 12: (23, 24),
 13: (25, 26),
 14: (27, 28),
 15: (29, 30),
 16: (31, 32),
 17: (33, 34),
 18: (35, 36),
 19: (37, 38),
 20: (39, 40),
 21: (41, 42),
 22: (43, 44),
 23: (45, 46),
 24: (47, 48),
 25: (49, 50),
 26: (51, 52),
 27: (53, 54)}

In [24]:
automaton2states[22]

(43, 44)

In [25]:
from itertools import combinations
import random
num_stars = 3
train_data = []
test_data = []
distractors = {}
for i in range(num_stars):
    choices = [i%27+1 for i in range((i+1)*9,(i+3)*9)]
    # generate all possible non-repeating triples from choices
    combinations_of_three = list(combinations(choices, 3))
    distractors[i+1]=combinations_of_three
train_dicts = []
for k,tcs in train_core.items():
    for tc in tcs:
        for dist in distractors[k]:
            train_dicts.append({'core':tc,'distractor':dist})

test_dicts = []
for k,tcs in test_core.items():
    for tc in tcs:
        rand_dists = random.sample(distractors[k], 5)
        for dist in rand_dists:
            test_dicts.append({'core':tc,'distractor':dist})
    

In [26]:
test_dicts

[{'core': (1, 5, 6), 'distractor': (15, 18, 19)},
 {'core': (1, 5, 6), 'distractor': (14, 19, 20)},
 {'core': (1, 5, 6), 'distractor': (15, 24, 26)},
 {'core': (1, 5, 6), 'distractor': (15, 16, 23)},
 {'core': (1, 5, 6), 'distractor': (21, 22, 23)},
 {'core': (2, 5, 7), 'distractor': (15, 19, 25)},
 {'core': (2, 5, 7), 'distractor': (10, 17, 18)},
 {'core': (2, 5, 7), 'distractor': (10, 14, 21)},
 {'core': (2, 5, 7), 'distractor': (10, 22, 27)},
 {'core': (2, 5, 7), 'distractor': (13, 19, 23)},
 {'core': (3, 5, 8), 'distractor': (12, 15, 16)},
 {'core': (3, 5, 8), 'distractor': (20, 24, 26)},
 {'core': (3, 5, 8), 'distractor': (11, 12, 14)},
 {'core': (3, 5, 8), 'distractor': (15, 24, 27)},
 {'core': (3, 5, 8), 'distractor': (11, 15, 24)},
 {'core': (4, 5, 9), 'distractor': (17, 23, 25)},
 {'core': (4, 5, 9), 'distractor': (21, 22, 26)},
 {'core': (4, 5, 9), 'distractor': (15, 16, 17)},
 {'core': (4, 5, 9), 'distractor': (10, 20, 26)},
 {'core': (4, 5, 9), 'distractor': (14, 16, 27)},


In [27]:
# a) one set of transitions where 3 automata are needed
# b) one set of transitions where only 2 automata are needed
# c) one set of transition where only 1 automaton is needed

# a) input format: [22:0, 23:0, 27:0, 1:0, 5:0, 11:0, 27:1]
# b) input format: [22:1, 23:0, 27:0, 1:0, 5:0, 11:0, 27:1]
# c) input format: [22:1, 23:1, 27:0, 1:0, 5:0, 11:0, 27:1]

# next create training samples by permuting the 6 first elements of the input and storing the position of the relevant automaton as output



In [28]:
from collections import defaultdict
import random
# TODO: přidat d), kde první dva stavy - druhý je dobře nastavený ale první je špatně
def create_transition_sets(test_dicts, automaton2states):
    result = defaultdict(list)

    for example in test_dicts:
        core = example['core'] # (22, 23, 27)
        distractor = example['distractor'] # (1, 4, 10)
        
        # a)
        input_dict_a = {}
        for node in core:
            input_dict_a[node] = automaton2states[node][0] # na 22 dám 43 (0), na 23 dám 45 (0), na 27 dám 53 (0) 
        for node in distractor:
            input_dict_a[node] = random.choice(automaton2states[node]) # na 1, 4, 10 dám náhodným výběrem stav, tzn. na 4 dám buď 7 nebo 8 (0 nebo 1)
        
        output_dict = {core[2]: automaton2states[core[2]][1]} # 3 automat v pořadí je finální, na něj dám stav 54 (1).
        result['a'].append({'in': input_dict_a, 'out': output_dict})
        
        # b)
        input_dict_b = {}
        for node in core:
            input_dict_b[node] = automaton2states[node][0] # stejné jako předtím
        input_dict_b[core[0]] = automaton2states[core[0]][1] # tady ale na v pořadí první automat 22 dám stav 44 (1)
        for node in distractor:
            input_dict_b[node] = input_dict_a[node]  # distractory mají stejné stavy pro a, b, c
        result['b'].append({'in': input_dict_b, 'out': output_dict})
        
        # c)
        input_dict_c = {}
        for node in core:
            input_dict_c[node] = automaton2states[node][0]
        input_dict_c[core[0]] = automaton2states[core[0]][1] # tady dávám na 22 stav 44 (1)
        input_dict_c[core[1]] = automaton2states[core[1]][1] # tady dávám na 23 stav 46 (1)
        for node in distractor:
            input_dict_c[node] = input_dict_a[node]  # distractory mají stejné stavy pro a, b, c
        result['c'].append({'in': input_dict_c, 'out': output_dict})

    return result

In [29]:
test_transition_sets = create_transition_sets(test_dicts, automaton2states)
train_transition_sets = create_transition_sets(train_dicts, automaton2states)
#sample:
for i, case in enumerate(['a', 'b', 'c']):
    print(f"{case}) {i+1}:")
    print(test_transition_sets[case][59])
    print()

a) 1:
{'in': {22: 43, 23: 45, 27: 53, 5: 10, 7: 13, 18: 36}, 'out': {27: 54}}

b) 2:
{'in': {22: 44, 23: 45, 27: 53, 5: 10, 7: 13, 18: 36}, 'out': {27: 54}}

c) 3:
{'in': {22: 44, 23: 46, 27: 53, 5: 10, 7: 13, 18: 36}, 'out': {27: 54}}



In [30]:
len(train_transition_sets["a"]) * 3

88128

In [31]:
# {'in': {22: 43, 23: 45, 27: 53, 4: 8, 12: 24, 16: 31}, 'out': {27: 54}}

# dict {input: [43, 53, 45, 8, 24, 31, 54], out: 0}

# {'in': {22: 44, 23: 45, 27: 53, 4: 8, 12: 24, 16: 31}, 'out': {27: 54}}

# dict {input: [43, 53, 45, 8, 24, 31, 54], out: 2}

# {'in': {22: 44, 23: 46, 27: 53, 4: 8, 12: 24, 16: 31}, 'out': {27: 54}}

# dict {[input: 43, 45, 8, 24, 31, 54], out: 1}

# 20 * 88128

In [32]:
train_transition_sets["c"][0]

{'in': {1: 2, 5: 10, 7: 13, 10: 20, 11: 21, 12: 23}, 'out': {7: 14}}

In [33]:
# vytvořit vektory
# 7 položek - poslední položka je cílové id_stavu 
# 6 předchozích jsou id stavů ale ty permutujeme (20 náhodných permutací)
# bude dict, kde 6 čísel bude permutovaných a 7 nechávám jak je - tj. dict {input: [vektor], output: [idx_pozice_automatu]}

In [34]:
import random
from itertools import permutations

def extract_state(transition_sets, num_permutations=20):
    result = []

    for case, t_set in transition_sets.items():
        for dicts in t_set:
            input_dict = dicts["in"]
            output_dict = dicts["out"]
            
            # prvních 6 stavů vytáhnu
            states = list(input_dict.values())[:6]
            # a vytáhnu výstupní stav
            output_state = list(output_dict.values())[0]
            
            # prozatím pro C, tady se vytáhne podle pozice stav, který je třeba změnit
            if case == 'c':
                target_state = states[2]
            else:
                continue
            # if case == 'b':
            #     target_state = states[1] 
            # if case == 'a':
            #     target_state = states[0] 
            
            # prvních 6 stavů - permutace
            permutations_list = list(permutations(states))
            # výběr 20
            selected_permutations = random.sample(permutations_list, num_permutations)
            # pro každou permutaci
            for perm in selected_permutations:
                # do listu dát tuto permutaci a výstupní stav
                input_list = list(perm) + [output_state]
                # do out hodit index cílového stavu
                orig_idx = input_list.index(target_state)
                out = orig_idx+55
                # dát do dictu
                result.append({'input': input_list, 'out': out, 'orig_idx': orig_idx})

    return result

data_train = extract_state(train_transition_sets)
data_test = extract_state(test_transition_sets)

In [37]:
data = {"train": data_train, "test": data_test}

In [38]:
import pickle
with open ("data.pkl", "wb") as f:
    pickle.dump(data, f)

In [39]:
data

{'train': [{'input': [13, 10, 20, 23, 21, 2, 14], 'out': 55, 'orig_idx': 0},
  {'input': [2, 20, 23, 21, 10, 13, 14], 'out': 60, 'orig_idx': 5},
  {'input': [23, 20, 10, 13, 21, 2, 14], 'out': 58, 'orig_idx': 3},
  {'input': [10, 13, 23, 21, 20, 2, 14], 'out': 56, 'orig_idx': 1},
  {'input': [2, 13, 10, 20, 21, 23, 14], 'out': 56, 'orig_idx': 1},
  {'input': [21, 13, 2, 20, 10, 23, 14], 'out': 56, 'orig_idx': 1},
  {'input': [20, 23, 2, 21, 10, 13, 14], 'out': 60, 'orig_idx': 5},
  {'input': [21, 2, 23, 20, 13, 10, 14], 'out': 59, 'orig_idx': 4},
  {'input': [21, 13, 20, 2, 23, 10, 14], 'out': 56, 'orig_idx': 1},
  {'input': [20, 21, 23, 10, 13, 2, 14], 'out': 59, 'orig_idx': 4},
  {'input': [10, 13, 20, 23, 2, 21, 14], 'out': 56, 'orig_idx': 1},
  {'input': [13, 21, 2, 20, 23, 10, 14], 'out': 55, 'orig_idx': 0},
  {'input': [13, 23, 10, 20, 21, 2, 14], 'out': 55, 'orig_idx': 0},
  {'input': [10, 13, 20, 21, 23, 2, 14], 'out': 56, 'orig_idx': 1},
  {'input': [21, 13, 20, 10, 2, 23, 14]